In [ ]:
HTML(read(open("style.css"), String))

# Advanced Board Struct

In [ ]:
using Chess
using NBInclude

In [ ]:
@nbinclude("Repetition.ipynb")

This notebook contains the `Advanced Board` (short: `AdvBoard`) struct which bundles additional information about the board to the board. As the `::Board` Class of Chess.jl does not contain all the information we need to the board this will be expanded to contain the following additional contents:

1. `score::Int64` is the static evaluation of the board in centipawn
1. `hash::UInt64`  is the hash of the board
1. `repCounter::Dict{UInt64, Int8}` is a Dictionary that counts how often a position as occured

In [ ]:
mutable struct AdvBoard
    state::Board
    score::Int64
    hash::UInt64
    repCounter::Dict{UInt64, Int8}
end

## Constructor for AdvBoard

In [ ]:
function AdvBoard(board::Board)::AdvBoard
    score = evaluate_position(board)
    hash  = zobrist_hash(board)
    repCounter = Dict{UInt64, Int8}()
    incrementHash!(repCounter, hash)
    return AdvBoard(board, score, hash, repCounter)
end

In [ ]:
function domoveAdv!(aBoard::AdvBoard, move::Move)
    oldscore:: Int64 = aBoard.score
    oldhash:: UInt64 = aBoard.hash
    aBoard.score = evaluate_move(aBoard, move) 
    aBoard.hash = zobrist_hash(aBoard.state, aBoard.hash, move) 
    undoinfo = domove!(aBoard.state, move)
    incrementHash!(aBoard.repCounter, aBoard.hash)
    return (oldscore, oldhash, undoinfo)
end

In [ ]:
function undomoveAdv!(ABoard::AdvBoard, undoinfo)
    decrementHash!(ABoard.repCounter, ABoard.hash)
    undomove!(ABoard.state, undoinfo[3])
    ABoard.score = undoinfo[1]
    ABoard.hash  = undoinfo[2]
end